# More on Type Systems and Polymorphism

These notes explore in greater depth how the type system and operational semantics of a host language can allow us to add features to an embedded language.

## Dependencies

In [10]:
# Presentation dependencies.
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib as mp
import matplotlib.pyplot as plt
from importlib import reload
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

## Working with Types in a Host Language (Python)

More information on the Python `typing` module can be found in its [documentation page](https://docs.python.org/3/library/typing.html). Note that while there is a built-in and endorsed way to *represent* types (i.e., abstract syntax trees for the language of Python types), there are no native, default, or "official" features for performing *type checking* on Python code.

In [2]:
# The typing module is included by default in Python 3.5 and higher.

def f(x: int) -> int:
    return x + x

print(f(123)) # No actual static or dynamic checking.
print(f('abc')) # No actual static or dynamic checking.

246
abcabc


The [mypy library](http://www.mypy-lang.org/) can annotate Python code with types, type-check a program statically (without running it), or introduce dynamic (i.e., at runtime) type checking.

Other community members have written libraries to perform type checking (such as [enforce](https://github.com/RussBaz/enforce)), but none are "official" and some appear to be dormant as of 2019.

In [11]:
from enforce import runtime_validation

@runtime_validation
def g(x: int) -> int:
    return 2 * x

try:
    g('abc')
except:
    print("A type error occurred.")

A type error occurred.


We present some additional examples below.

In [8]:
from typing import Dict, Tuple, Sequence

def repeat(si: Tuple[str, int]) -> str:
    (s, i) = si
    return s * i

print(repeat(('xyz', 3)))

xyzxyzxyz


In [7]:
from typing import NewType

UserName = NewType('UserName', str)

def confirm(s: UserName) -> bool:
    return s == 'Alice'

print(confirm('Alice'))
print(confirm('Bob'))
print(confirm(123)) # No type checking by default.

True
False
False


In [6]:
from typing import Sequence, TypeVar, Union

T = TypeVar('T')
S = Union[int, str]

def first(xs: Sequence[T]) -> T:
    return xs[0]

first([1,2,3])
first(['a', 'b', 'c'])

'a'

## Polymorphism

There are at least two distinct types of polymorphism: *parametric polymorphism* and *ad hoc polymorphism*.

Operator overloading is an example of ad hoc polymorphism.

### Ad Hoc Polymorphism in Python

In the example below, we use Python's support for ad hoc polymorphism via operator overloading to create a new data structure. In this way, we can define our own semantics for built-in Python operators (and other constructs, such as the indexing notation in expressions such as `example[index]`) when those operators are applied to instances of our data structure. 

In [12]:
class Zoo():
    def __init__(self, *animals):
        self.animals = animals
    
    def __hash__(self):
        return hash("constant")
    
    def __add__(self, other):
        return Zoo(*(self.animals + other.animals))

    def __lt__(self, other):
        return set(self.animals).issubset(set(other.animals))

    def __getitem__(self, animal):
        return animal in self.animals

    def __call__(self, animal):
        self.animals += tuple([animal])

    def __len__(self):
        return len(self.animals)
    
    def __str__(self):
        return "Zoo" + str(self.animals)

    def __repr__(self):
        return str(self)
    
    def __matmul__(self, other):
        return Zoo(*(tuple(set(self.animals) - set(other.animals))))
    
z1 = Zoo("Giraffe")
z2 = Zoo("Zebra", "Giraffe")
print(z1 + z2)
print(z2 < z1)
print(z1["Giraffe"])
print(z1["Zebra"])
z1("Lion")
print(z1)

Zoo('Giraffe', 'Zebra', 'Giraffe')
False
True
False
Zoo('Giraffe', 'Lion')


The [SymPy](https://www.sympy.org/en/index.html) library itself uses both reflection (in order to parse strings that contain a concrete syntax representation of a polynomial) and ad hoc polymorphism (in order to allow users to use built-in Python operators to assemble objects that represent larger polynomials).

In [4]:
import ast
from sympy import sympify

e1 = sympify("x**2 + (x + 2)")
e2 = sympify("x**3 + 2")
e3 = e1 * e2
2 * e3

2*(x**3 + 2)*(x**2 + x + 2)

The [Z3 Theorem Prover](https://pypi.org/project/z3-solver/) is another example of effective use of ad hoc polymorphism to create an embedded language for representing constraint sets. The language allows users to create formula objects 

In [5]:
import z3

(x1,x2,x3,x4,x5,x6,x7) = [z3.Real('x'+str(i)) for i in range(1,8)]

S = z3.Solver()

# Only allow non-negative flows.
for x in (x1,x2,x3,x4,x5,x6,x7):
    S.add(x >= 0)
    
# Edge capacity constraints.
S.add(x2 <= 7, x3 <= 8, x4 <= 6)
S.add(x5 <= 3, x6 <= 4, x7 <= 5)

# Constraints derived from graph topology.
S.add(x1 == x2+x3, x2 == x4+x5, x3+x4 == x6, x5+x6 == x7)

S.add(x1 > 0) # We want a positive flow.

print(S.check())
print(S.model())

sat
[x3 = 0,
 x2 = 1/2,
 x1 = 1/2,
 x4 = 1/2,
 x5 = 0,
 x6 = 1/2,
 x7 = 1/2]


This is the end of the document.